#1.Data Load

In [1]:
import warnings
warnings.filterwarnings('ignore')

# 라이브러리

import pandas as pd
import random
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import VotingClassifier
import lightgbm as lgbm
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier

In [2]:
# Seed 고정

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) 

In [3]:
# 구글드라이브 활용

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/sample_data/

/content/sample_data


In [5]:
!unzip -qq '/content/drive/MyDrive/project_study_KKUL/009_LG/open (1).zip'

In [27]:
test = pd.read_csv('/content/sample_data/test.csv')
train = pd.read_csv('/content/sample_data/train.csv')
sub = pd.read_csv('/content/sample_data/sample_submission.csv')

#2.데이터 확인

##1) train data 확인

* 1132 rows x 3331 columns

In [13]:
train.info

<bound method DataFrame.info of       PRODUCT_ID  Y_Class  Y_Quality     LINE PRODUCT_CODE  X_1    X_2  X_3  \
0     TRAIN_0000        1   0.531957  T100304         T_31  2.0   95.0  0.0   
1     TRAIN_0001        1   0.531105  T100306         T_31  2.0   96.0  0.0   
2     TRAIN_0002        1   0.532292  T100306         T_31  2.0   95.0  0.0   
3     TRAIN_0003        1   0.528141  T100306         T_31  2.0   87.0  0.0   
4     TRAIN_0004        1   0.532289  T100306         T_31  2.0   95.0  0.0   
...          ...      ...        ...      ...          ...  ...    ...  ...   
1127  TRAIN_1127        0   0.516681  T050304         A_31  NaN    NaN  NaN   
1128  TRAIN_1128        1   0.530413  T100304         T_31  2.0  102.0  0.0   
1129  TRAIN_1129        1   0.533311  T100306         T_31  1.0   88.0  0.0   
1130  TRAIN_1130        0   0.513135  T010306         A_31  NaN    NaN  NaN   
1131  TRAIN_1131        1   0.527129  T100304         T_31  2.0  101.0  0.0   

       X_4   X_5  .

In [7]:
train.head()

,PRODUCT_ID,Y_Class,Y_Quality,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,...,X_3317,X_3318,X_3319,X_3320,X_3321,X_3322,X_3323,X_3324,X_3325,X_3326
0,TRAIN_0000,1,0.531957,T100304,T_31,2.0,95.0,0.0,45.0,10.0,...,0.000008,0.000003,0.191408,0.000008,0.001210,0.000021,0.000003,0.000002,0.189,0.000006
1,TRAIN_0001,1,0.531105,T100306,T_31,2.0,96.0,0.0,45.0,10.0,...,0.000008,0.000003,0.188993,0.000032,0.000644,0.000041,0.000002,0.000003,0.185,0.000029
2,TRAIN_0002,1,0.532292,T100306,T_31,2.0,95.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRAIN_0003,1,0.528141,T100306,T_31,2.0,87.0,0.0,45.0,10.0,...,0.000007,0.000003,0.189424,0.000034,0.000678,0.000043,0.000004,0.000003,0.188,0.000031
4,TRAIN_0004,1,0.532289,T100306,T_31,2.0,95.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### LINE, PRODUCT_CODE 별 데이터 확인

In [8]:
cols = ['LINE', 'PRODUCT_CODE']

for col in cols:
  def count_into_list(lst):
    answer = dict()
    for num in train[col]:
      if num not in answer.keys():
        answer[num] = 1
      else:
        answer[num] += 1
    return answer

  count_train = count_into_list(train[col])
  print(f'-- {col} --')
  print(count_train, '\n')

-- LINE --
{'T100304': 404, 'T100306': 434, 'T010305': 66, 'T050307': 66, 'T050304': 86, 'T010306': 76} 

-- PRODUCT_CODE --
{'T_31': 830, 'A_31': 294, 'O_31': 8} 



### PIVOT
* PRODUCT_CODE, LINE을 기준으로 PIVOT Table 만들기
* 이때, PRODUCT_CODE 별로 해당하는 라인이 각각 다름을 확인할 수 있음

In [10]:
train_pivot = pd.pivot_table(train, index = 'PRODUCT_CODE', columns = 'LINE', values = 'Y_Class', aggfunc = 'count')
train_pivot

LINE,T010305,T010306,T050304,T050307,T100304,T100306
PRODUCT_CODE,,,,,,
A_31,66.0,76.0,86.0,66.0,NaN,NaN
O_31,NaN,NaN,NaN,NaN,4.0,4.0
T_31,NaN,NaN,NaN,NaN,400.0,430.0


## 2) test data 확인

* 535 rows x 3329 columns

In [14]:
test.info

<bound method DataFrame.info of     PRODUCT_ID     LINE PRODUCT_CODE  X_1    X_2  X_3   X_4   X_5  X_6   X_7  \
0     TEST_000  T100304         T_31  2.0   94.0  0.0  45.0  11.0  0.0  45.0   
1     TEST_001  T100306         T_31  3.0   89.0  0.0  45.0  10.0  0.0  51.0   
2     TEST_002  T100304         T_31  2.0  100.0  0.0  45.0  10.0  0.0  45.0   
3     TEST_003  T100306         T_31  2.0   91.0  0.0  45.0  10.0  0.0  51.0   
4     TEST_004  T100306         T_31  2.0   87.0  0.0  45.0  10.0  0.0  51.0   
..         ...      ...          ...  ...    ...  ...   ...   ...  ...   ...   
530   TEST_530  T100306         T_31  1.0   95.0  0.0  45.0  10.0  0.0  51.0   
531   TEST_531  T100306         T_31  2.0   91.0  0.0  45.0  10.0  0.0  53.0   
532   TEST_532  T100304         T_31  2.0   93.0  0.0  45.0  10.0  0.0  45.0   
533   TEST_533  T100306         T_31  2.0   93.0  0.0  45.0  10.0  0.0  51.0   
534   TEST_534  T100306         T_31  1.0   90.0  0.0  45.0  10.0  0.0  50.0   

     ..

In [11]:
test.head()

,PRODUCT_ID,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,...,X_3317,X_3318,X_3319,X_3320,X_3321,X_3322,X_3323,X_3324,X_3325,X_3326
0,TEST_000,T100304,T_31,2.0,94.0,0.0,45.0,11.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TEST_001,T100306,T_31,3.0,89.0,0.0,45.0,10.0,0.0,51.0,...,0.000007,0.000004,0.189134,0.000032,0.000664,0.000041,0.000004,0.000003,0.187,0.00003
2,TEST_002,T100304,T_31,2.0,100.0,0.0,45.0,10.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TEST_003,T100306,T_31,2.0,91.0,0.0,45.0,10.0,0.0,51.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TEST_004,T100306,T_31,2.0,87.0,0.0,45.0,10.0,0.0,51.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### LINE, PRODUCT_CODE 별 데이터 확인

In [12]:
cols = ['LINE', 'PRODUCT_CODE']

for col in cols:
  def count_into_list(lst):
    answer = dict()
    for num in test[col]:
      if num not in answer.keys():
        answer[num] = 1
      else:
        answer[num] += 1
    return answer
    
  count_test = count_into_list(test[col])
  print(f'-- {col} --')
  print(count_test, '\n')

-- LINE --
{'T100304': 213, 'T100306': 276, 'T050307': 26, 'T050304': 5, 'T010306': 8, 'T010305': 7} 

-- PRODUCT_CODE --
{'T_31': 487, 'A_31': 46, 'O_31': 2} 



# 3.데이터 전처리

## 1) 불필요한 열 삭제

In [28]:
train_x = train.drop(columns=['PRODUCT_ID', 'Y_Class', 'Y_Quality'])
train_y = train['Y_Class']

test_x = test.drop(columns=['PRODUCT_ID'])

## 2) 라벨인코딩

In [29]:
# 라벨 인코딩 전 train data
train_x['PRODUCT_CODE'].unique()

array(['T_31', 'A_31', 'O_31'], dtype=object)

In [30]:
# 라벨 인코딩 전 test data
test_x['PRODUCT_CODE'].unique()

array(['T_31', 'A_31', 'O_31'], dtype=object)

In [33]:
# qualitative to quantitative

qual_col = ['PRODUCT_CODE', 'LINE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test[i]) 
print('Done.')

Done.


In [34]:
# 라벨 인코딩 후 train data
train_x['PRODUCT_CODE'].unique()

array([2, 0, 1])

In [35]:
# 라벨 인코딩 후 test data
test_x['PRODUCT_CODE'].unique()

array([5, 3, 4])

## 3) 정규화

In [40]:
# 정규화 전 train data
train_x.loc[1128:1129, 'X_1':'X_8']

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8
1128,2.0,102.0,0.0,45.0,11.0,0.0,45.0,10.0
1129,1.0,88.0,0.0,45.0,10.0,0.0,51.0,10.0


In [42]:
# 정규화 전 test data
test_x.loc[530:534, 'X_1':'X_8']

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8
530,1.0,95.0,0.0,45.0,10.0,0.0,51.0,10.0
531,2.0,91.0,0.0,45.0,10.0,0.0,53.0,10.0
532,2.0,93.0,0.0,45.0,10.0,0.0,45.0,10.0
533,2.0,93.0,0.0,45.0,10.0,0.0,51.0,10.0
534,1.0,90.0,0.0,45.0,10.0,0.0,50.0,10.0


In [44]:
from sklearn.preprocessing import MinMaxScaler

x_col = train.columns[train.columns.str.contains('X')].tolist()
scaler = MinMaxScaler()
  
scaler.fit(train_x[x_col])
  
train_x[x_col] = scaler.transform(train_x[x_col])
test_x[x_col] = scaler.transform(test_x[x_col])

In [45]:
# 정규화 후 train data
train_x.loc[1128:1129, 'X_1':'X_8']

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8
1128,0.006536,1.000000,0.0,0.0,1.0,0.0,0.000000,0.0
1129,0.000000,0.066667,0.0,0.0,0.0,0.0,0.272727,0.0


In [46]:
# 정규화 후 test data
test_x.loc[530:534, 'X_1':'X_8']

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8
530,0.000000,0.533333,0.0,0.0,0.0,0.0,0.272727,0.0
531,0.006536,0.266667,0.0,0.0,0.0,0.0,0.363636,0.0
532,0.006536,0.400000,0.0,0.0,0.0,0.0,0.000000,0.0
533,0.006536,0.400000,0.0,0.0,0.0,0.0,0.272727,0.0
534,0.000000,0.200000,0.0,0.0,0.0,0.0,0.227273,0.0


## 4) 결측값 'train_x' 평균으로 채우기

In [48]:
# 결측값 대체 전 train data
train_x.loc[:5, 'X_3317':'X_3319']

,X_3317,X_3318,X_3319
0,0.654045,0.002252,0.531399
1,0.604131,0.144144,0.027793
2,NaN,NaN,NaN
3,0.495697,0.191441,0.117571
4,NaN,NaN,NaN
5,0.407917,0.200450,0.252396


In [50]:
# 결측값 대체 전 test data
test_x.loc[:5, 'X_3317':'X_3319']

,X_3317,X_3318,X_3319
0,NaN,NaN,NaN
1,0.495697,0.20045,0.057182
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN


In [51]:
# data leakage를 방지하기 위해 train_x의 평균으로 test_x의 결측치를 채움
train_X = train_x.fillna(train_x.mean())
test_x = test_x.fillna(train_x.mean())

In [53]:
# 결측값 대체 후 train data
train_X.loc[:5, 'X_3317':'X_3319']

,X_3317,X_3318,X_3319
0,0.654045,0.002252,0.531399
1,0.604131,0.144144,0.027793
2,0.486506,0.233557,0.315822
3,0.495697,0.191441,0.117571
4,0.486506,0.233557,0.315822
5,0.407917,0.200450,0.252396


In [54]:
# 결측값 대체 후 test data
test_x.loc[:5, 'X_3317':'X_3319']

,X_3317,X_3318,X_3319
0,0.486506,0.233557,0.315822
1,0.495697,0.200450,0.057182
2,0.486506,0.233557,0.315822
3,0.486506,0.233557,0.315822
4,0.486506,0.233557,0.315822
5,0.486506,0.233557,0.315822


## 5) 중복 값인 열 삭제하기

In [55]:
# train_x 행, 열 바꾸기

train_xt = train_X.transpose()
train_xt

,0,1,2,3,4,5,6,7,8,9,...,1122,1123,1124,1125,1126,1127,1128,1129,1130,1131
LINE,4.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,4.000000,5.000000,4.000000,...,4.000000,5.000000,4.000000,5.000000,4.000000,2.000000,4.000000,5.000000,1.000000,4.000000
PRODUCT_CODE,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,0.000000,2.000000,2.000000,0.000000,2.000000
X_1,0.006536,0.006536,0.006536,0.006536,0.006536,0.006536,0.006536,0.000000,0.013072,0.307190,...,0.000000,0.006536,0.006536,0.006536,0.006536,0.011473,0.006536,0.000000,0.011473,0.006536
X_2,0.533333,0.600000,0.533333,0.000000,0.533333,0.533333,0.533333,0.666667,0.400000,0.666667,...,0.933333,0.600000,0.533333,0.000000,0.400000,0.530231,1.000000,0.066667,0.530231,0.933333
X_3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X_3322,0.068966,0.758621,0.463106,0.844828,0.463106,0.720690,0.463106,0.463106,0.693103,0.310345,...,0.463106,0.463106,0.463106,0.463106,0.463106,0.463106,0.463106,0.463106,0.463106,0.110345
X_3323,0.374749,0.174349,0.330111,0.468938,0.330111,0.571142,0.330111,0.330111,0.232465,0.995992,...,0.330111,0.330111,0.330111,0.330111,0.330111,0.330111,0.330111,0.330111,0.330111,0.414830
X_3324,0.382716,0.703036,0.583531,0.759760,0.583531,0.843177,0.583531,0.583531,0.723056,0.646313,...,0.583531,0.583531,0.583531,0.583531,0.583531,0.583531,0.583531,0.583531,0.583531,0.289289
X_3325,0.500000,0.000000,0.448529,0.375000,0.448529,0.375000,0.448529,0.448529,0.500000,0.750000,...,0.448529,0.448529,0.448529,0.448529,0.448529,0.448529,0.448529,0.448529,0.448529,0.625000


In [56]:
 # train_x 중복 행 삭제하기

train_xtd = train_xt.drop_duplicates(keep = False)
train_xtd

,0,1,2,3,4,5,6,7,8,9,...,1122,1123,1124,1125,1126,1127,1128,1129,1130,1131
LINE,4.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,4.000000,5.000000,4.000000,...,4.000000,5.000000,4.000000,5.000000,4.000000,2.000000,4.000000,5.000000,1.000000,4.000000
PRODUCT_CODE,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,0.000000,2.000000,2.000000,0.000000,2.000000
X_1,0.006536,0.006536,0.006536,0.006536,0.006536,0.006536,0.006536,0.000000,0.013072,0.307190,...,0.000000,0.006536,0.006536,0.006536,0.006536,0.011473,0.006536,0.000000,0.011473,0.006536
X_2,0.533333,0.600000,0.533333,0.000000,0.533333,0.533333,0.533333,0.666667,0.400000,0.666667,...,0.933333,0.600000,0.533333,0.000000,0.400000,0.530231,1.000000,0.066667,0.530231,0.933333
X_4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.005967,0.000000,0.000000,0.005967,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X_3322,0.068966,0.758621,0.463106,0.844828,0.463106,0.720690,0.463106,0.463106,0.693103,0.310345,...,0.463106,0.463106,0.463106,0.463106,0.463106,0.463106,0.463106,0.463106,0.463106,0.110345
X_3323,0.374749,0.174349,0.330111,0.468938,0.330111,0.571142,0.330111,0.330111,0.232465,0.995992,...,0.330111,0.330111,0.330111,0.330111,0.330111,0.330111,0.330111,0.330111,0.330111,0.414830
X_3324,0.382716,0.703036,0.583531,0.759760,0.583531,0.843177,0.583531,0.583531,0.723056,0.646313,...,0.583531,0.583531,0.583531,0.583531,0.583531,0.583531,0.583531,0.583531,0.583531,0.289289
X_3325,0.500000,0.000000,0.448529,0.375000,0.448529,0.375000,0.448529,0.448529,0.500000,0.750000,...,0.448529,0.448529,0.448529,0.448529,0.448529,0.448529,0.448529,0.448529,0.448529,0.625000


In [57]:
# 다시 train_x 행, 열 바꾸기 

train_xtt = train_xtd.transpose()
train_xtt

,LINE,PRODUCT_CODE,X_1,X_2,X_4,X_5,X_7,X_8,X_9,X_11,...,X_3317,X_3318,X_3319,X_3320,X_3321,X_3322,X_3323,X_3324,X_3325,X_3326
0,4.0,2.0,0.006536,0.533333,0.000000,0.000000,0.000000,0.000000,0.0000,0.530547,...,0.654045,0.002252,0.531399,0.041652,0.849850,0.068966,0.374749,0.382716,0.500000,0.072881
1,5.0,2.0,0.006536,0.600000,0.000000,0.000000,0.363636,0.000000,1.0000,0.915327,...,0.604131,0.144144,0.027793,0.872926,0.000000,0.758621,0.174349,0.703036,0.000000,0.877966
2,5.0,2.0,0.006536,0.533333,0.000000,0.000000,0.681818,0.000000,1.0000,0.735263,...,0.486506,0.233557,0.315822,0.480294,0.491698,0.463106,0.330111,0.583531,0.448529,0.470894
3,5.0,2.0,0.006536,0.000000,0.000000,0.000000,0.363636,0.000000,1.0000,0.482315,...,0.495697,0.191441,0.117571,0.964702,0.051051,0.844828,0.468938,0.759760,0.375000,0.949153
4,5.0,2.0,0.006536,0.533333,0.000000,0.000000,0.272727,0.000000,1.0000,0.798499,...,0.486506,0.233557,0.315822,0.480294,0.491698,0.463106,0.330111,0.583531,0.448529,0.470894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,2.0,0.0,0.011473,0.530231,0.005967,0.335322,0.163051,0.027446,0.5179,0.659740,...,0.486506,0.233557,0.315822,0.480294,0.491698,0.463106,0.330111,0.583531,0.448529,0.470894
1128,4.0,2.0,0.006536,1.000000,0.000000,1.000000,0.000000,0.000000,0.0000,0.918542,...,0.486506,0.233557,0.315822,0.480294,0.491698,0.463106,0.330111,0.583531,0.448529,0.470894
1129,5.0,2.0,0.000000,0.066667,0.000000,0.000000,0.272727,0.000000,1.0000,0.336549,...,0.486506,0.233557,0.315822,0.480294,0.491698,0.463106,0.330111,0.583531,0.448529,0.470894
1130,1.0,0.0,0.011473,0.530231,0.005967,0.335322,0.163051,0.027446,0.5179,0.659740,...,0.486506,0.233557,0.315822,0.480294,0.491698,0.463106,0.330111,0.583531,0.448529,0.470894


> train 열과 똑같은 test 열 남기기

In [58]:
## train의 기존 columns을 리스트 변수로 설정
train_col_origin = train.columns[train.columns.str.contains('X')].tolist()

## 중복 열이 제거된 train의 남은 columns을 리스트 변수로 설정
train_col = train_xtt.columns[train_xtt.columns.str.contains('X')].tolist()

## train 열 중 삭제 된 columns 구하기
train_col_main = [x for x in train_col_origin if x not in train_col]

In [59]:
## 삭제된 columns을 test에서도 삭제하기
for col in train_col_main:
  test_x = test_x.drop(columns = col)

# Modeling

In [ ]:
train_xtt.shape, train_y.shape, test_x.shape

((1132, 1381), (1132,), (535, 1381))

In [ ]:
et_cls = ExtraTreesClassifier(n_estimators = 500, min_samples_leaf = 4, 
                              min_samples_split= 7, max_features=1381, random_state = 2045)
et_cls.fit(train_xtt, train_y)

ExtraTreesClassifier(max_features=1381, min_samples_leaf=4, min_samples_split=7,
                     n_estimators=500, random_state=2045)

In [ ]:
pred = et_cls.predict(test_x)
sub['Y_Class'] = pred

sub.to_csv('./answer.csv', index = False)

In [ ]:
pred

array([1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1,
       2, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 0, 1, 2, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 0, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2,
       1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 0, 1, 1, 1, 1, 2, 0, 1, 1, 0, 1,
       1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1,
       2, 2, 0, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,